1. Setup Environment 🔽

This cell downloads the initial setup.py script from your fork and then executes it, instructing the setup process to pull all subsequent project files from your repository.

In [ ]:
# @title 1. Setup Environment (Final Path Correction)
import os
import sys
from pathlib import Path

# --- DYNAMIC PLATFORM DETECTION & PATH SETUP ---
# This logic ensures the notebook and the scripts it runs agree on the correct home directory.
def get_platform_home():
    """Determines the correct project HOME based on the runtime environment."""
    if 'google.colab' in sys.modules:
        print("Platform: Google Colab")
        return Path('/content')
    if os.path.exists('/kaggle'):
        print("Platform: Kaggle")
        return Path('/kaggle/working')
    if os.environ.get('LIGHTNING_AI') or os.path.exists('/teamspace'):
        print("Platform: Lightning AI")
        base_path = Path('/teamspace/studios/this_studio')
        if not base_path.exists():
            base_path = Path.home() / 'workspace'
        return base_path
    # Fallback for local or unrecognized environments
    print("Platform: Local / Unknown")
    return Path.cwd()

# Define the single source of truth for the project's home directory for this entire session
PROJECT_HOME = get_platform_home()
print(f"✅ Project HOME directory set to: {PROJECT_HOME}")

# --- Configuration ---
lang = 'en'
branch = 'main'
fork_repo = 'drf0rk/AnxietyLightning'

# --- Script Execution ---
anxety_path = PROJECT_HOME / 'ANXETY'
scripts_dir_temp = anxety_path / 'scripts'
setup_script_path = scripts_dir_temp / 'setup.py'

os.makedirs(scripts_dir_temp, exist_ok=True)

setup_url = f'https://raw.githubusercontent.com/{fork_repo}/{branch}/scripts/setup.py'
print(f"🔄 Downloading setup script from {fork_repo}...")
!curl -sLo {setup_script_path} {setup_url}
print("✅ Setup script downloaded.")

# Run the setup script to detect the platform and create settings.json
%run {setup_script_path} --lang={lang} --branch={branch} --fork={fork_repo}

# --- Set Definitive Paths for Subsequent Cells ---
print("\n✅ Setting up definitive paths for the notebook session...")
modules_path = anxety_path / 'modules'
if str(modules_path) not in sys.path:
    sys.path.insert(0, str(modules_path))

import json_utils as js
settings_path = anxety_path / 'settings.json'

# --- THIS IS THE CORRECTED LOGIC ---
# Read the base 'scr_path' (e.g., /content/ANXETY) and then append '/scripts' to it.
base_scr_path = Path(js.read(settings_path, 'ENVIRONMENT.scr_path'))
scripts_dir = base_scr_path / 'scripts'
# --- END CORRECTION ---

print(f"✔️ Scripts directory for this session is set to: {scripts_dir}")

2. Widgets 🔽

This cell will load the interactive widgets. Since setup.py (executed in the previous cell) has already downloaded the patched widgets-en.py (or widgets-ru.py) to scripts_dir, this command will now use your modified version with the LoRA dropdown.

In [ ]:
# @title 2. Widgets
# This cell uses the 'scripts_dir' and 'lang' variables that were correctly defined in Cell 1.

# The f-string constructs the full path to the script.
# We print the path first for easy debugging to ensure it's correct.
run_path = f"{scripts_dir}/{lang}/widgets-{lang}.py"

print(f"✅ Attempting to run script from: {run_path}")
%run {run_path}

3. Downloading 🔽

This cell initiates the downloading process for models, VAEs, extensions, and other necessary components. It will use the patched downloading-en.py (or downloading-ru.py) from your fork, ensuring downloads go to your centralized model storage location.

In [ ]:
# @title 3. Downloading
# This cell uses the 'scripts_dir' and 'lang' variables defined in Cell 1.

# The f-string constructs the full path to the script.
# We print the path first for easy debugging.
run_path = f"{scripts_dir}/{lang}/downloading-{lang}.py"

print(f"✅ Attempting to run script from: {run_path}")
%run {run_path}

4. Start 🔽

This cell launches the Stable Diffusion WebUI. It will execute the patched launch.py script from your fork, applying any platform-specific optimizations and arguments you've included.

In [ ]:
# @title 4. Start (Corrected Path Logic)
from pathlib import Path
import sys

# Add the modules path to ensure json_utils can be imported
anxety_path = Path.home() / 'ANXETY'
modules_path = anxety_path / 'modules'
if str(modules_path) not in sys.path:
    sys.path.insert(0, str(modules_path))

import json_utils as js

# Read the correct scripts_dir path that was saved by setup.py
settings_path = anxety_path / 'settings.json'
scripts_dir = Path(js.read(settings_path, 'ENVIRONMENT.scr_path'))

print(f"✅ Running launch script from: {scripts_dir}")
# Launch the WebUI. The -l flag provides more detailed logging for tunnels.
%run {scripts_dir}/launch.py -l

Utilities
5. Run Cleanup Utility 🔽

This cell runs the new Cleanup Utility GUI script, allowing you to manage your environment. This script is downloaded from your fork via the initial setup.

In [ ]:
# @title 5. Run Cleanup Utility

from pathlib import Path

# This cell is now platform-agnostic.
scripts_dir = Path.home() / 'ANXETY' / 'scripts'

# Run the cleanup utility GUI
%run {scripts_dir}/auto-cleaner.py

In [ ]:
from pathlib import Path
import shutil
import os
import sys

# --- Start of Directory Management ---
BASE_DIR = Path("/teamspace/studios/this_studio")
if os.getcwd() != str(BASE_DIR):
    print(f"🔄 Changing directory from {os.getcwd()} to {BASE_DIR}")
    os.chdir(BASE_DIR)
# --- End of Directory Management ---

# --- VERY IMPORTANT WARNING ---
# This cell will DELETE almost everything in your Lightning AI instance.
# It will only preserve this notebook file and 'main.py'.
# Ensure you understand what is being deleted before running.
# This operation is IRREVERSIBLE.

print("!!! DANGER: YOU ARE ABOUT TO DELETE ALMOST ALL FILES !!!")
print("!!! PLEASE READ CAREFULLY BEFORE PROCEEDING !!!")
print("\nThis cell will delete all folders and files in your current studio instance, EXCEPT:")
print(" - This notebook file (e.g., 'LightningAnxiety (1) (1) (2).ipynb')")
print(" - The 'main.py' file (if it exists in the root)")
print("\nTHIS WILL REQUIRE YOU TO RERUN THE ENTIRE NOTEBOOK FROM THE FIRST CELL FOR A FRESH START.")
print("If you have any custom files you wish to keep, MOVE THEM OUTSIDE THIS STUDIO INSTANCE NOW.")
print("Proceed only if you want a completely blank studio environment.")
print("\nType 'YES_DELETE_ALL' (case-sensitive) to confirm deletion and execute, then press Enter.")
print("Anything else will abort the operation.")

confirmation = input("Confirmation: ")

if confirmation.strip() == "YES_DELETE_ALL": # Case-sensitive comparison
    # Get the home/studio path
    # On Lightning AI, this is typically /teamspace/studios/this_studio
    HOME_PATH = Path.home()

    # Get the current notebook's filename
    # Use a more robust way to find the notebook file name, as __file__ is not always reliable in notebooks
    notebook_filename = "LightningAnxiety (1) (1) (2).ipynb" # Explicitly set your notebook filename here for reliability
    notebook_path = HOME_PATH / notebook_filename

    main_py_path = HOME_PATH / "main.py"

    # Define items to EXCLUDE from deletion
    EXCLUDE_LIST = [
        notebook_path,
        main_py_path
    ]

    print(f"\n--- Starting Comprehensive Deletion in {HOME_PATH} ---")
    deleted_count = 0
    skipped_count = 0

    for item in HOME_PATH.iterdir():
        # Convert item to absolute path for consistent comparison with EXCLUDE_LIST
        abs_item = item.resolve()
        if abs_item in EXCLUDE_LIST:
            print(f"ℹ️ Skipping protected item: {item.name}")
            skipped_count += 1
            continue

        print(f"🗑️ Attempting to delete: {item.name} ({item})...")
        try:
            if item.is_dir():
                shutil.rmtree(item)
            else:
                item.unlink() # Delete file
            print(f"✅ Successfully deleted: {item.name}")
            deleted_count += 1
        except Exception as e:
            print(f"❌ Error deleting {item.name} ({item}): {e}")

    print("\n--- Comprehensive Cleanup Process Complete ---")
    print(f"Summary: {deleted_count} items deleted, {skipped_count} items skipped (protected).")
    print("Please restart your runtime and run the notebook from the first cell for a fresh start.")
else:
    print("\nOperation aborted. No folders were deleted.\n")

# --- Ensure we are back in BASE_DIR at the end of the cell ---
if os.getcwd() != str(BASE_DIR):
    os.chdir(BASE_DIR)
# --- End of Directory Management ---
